In [10]:
import requests
import folium
from IPython.display import display

# OpenWeatherMap API Key
API_KEY="a12450e4893d948621b5ff39738277d7"  

# Major cities with latitude and longitude
cities = {
    "Delhi": [28.7041, 77.1025],
    "Mumbai": [19.0760, 72.8777],
    "Kolkata": [22.5726, 88.3639],
    "Bangalore": [12.9716, 77.5946],
    "Chennai": [13.0827, 80.2707],
    "Hyderabad": [17.3850, 78.4867],
    "Ahmedabad": [23.0225, 72.5714],
    "Pune": [18.5204, 73.8567],
    "Jaipur": [26.9124, 75.7873],
    "Lucknow": [26.8467, 80.9462],
}

# Function to fetch AQI and calculate concentration values
def get_aqi(city, lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        components = data['list'][0]['components']  # Pollutant details

        # Extract pollutant values (μg/m³ or ppm)
        pm25 = components['pm2_5']
        pm10 = components['pm10']
        co = components['co']
        no2 = components['no2']
        o3 = components['o3']
        so2 = components['so2']

        # For simplicity, we assume PM2.5 is the most critical factor
        # You can extend the calculation based on other pollutants as needed
        # AQI calculation formula (simplified, using PM2.5 concentration)
        if pm25 <= 12:
            aqi = (pm25 / 12) * 50  # Scaling to AQI range 0-50
        elif pm25 <= 35.4:
            aqi = 51 + ((pm25 - 12.1) / (35.4 - 12.1)) * (100 - 51)
        elif pm25 <= 55.4:
            aqi = 101 + ((pm25 - 35.5) / (55.4 - 35.5)) * (150 - 101)
        elif pm25 <= 150.4:
            aqi = 151 + ((pm25 - 55.5) / (150.4 - 55.5)) * (200 - 151)
        else:
            aqi = 201 + ((pm25 - 150.5) / (250.4 - 150.5)) * (300 - 201)

        # Assign the correct AQI category and range
        if aqi <= 50:
            aqi_rating = "Good"
            aqi_range = "0-50"
        elif aqi <= 100:
            aqi_rating = "Fair"
            aqi_range = "51-100"
        elif aqi <= 150:
            aqi_rating = "Moderate"
            aqi_range = "101-150"
        elif aqi <= 200:
            aqi_rating = "Poor"
            aqi_range = "151-200"
        else:
            aqi_rating = "Very Poor"
            aqi_range = "201+"

        # Display the results
        print(f"\nCity: {city}")
        print(f"Real-Time AQI: {int(aqi)} ({aqi_rating}) - Range: {aqi_range}")
        print("Pollutant Concentrations (μg/m³):")
        print(f"CO: {components['co']}, NO2: {components['no2']}, O3: {components['o3']}")
        print(f"PM2.5: {components['pm2_5']}, PM10: {components['pm10']}, SO2: {components['so2']}")

        # Visualize the data on a map using Folium
        map_center = [lat, lon]
        city_map = folium.Map(location=map_center, zoom_start=12)

        # Add a marker for the city with AQI info
        folium.Marker(
            location=map_center,
            popup=f"City: {city}\nAQI: {aqi_rating}\nCO: {components['co']} µg/m³\nNO2: {components['no2']} µg/m³\nO3: {components['o3']} µg/m³\nPM2.5: {components['pm2_5']} µg/m³\nPM10: {components['pm10']} µg/m³\nSO2: {components['so2']} µg/m³",
            icon=folium.Icon(color="blue", icon="info-sign"),
        ).add_to(city_map)

        # Add a circle around the location indicating pollution intensity
        folium.Circle(
            location=map_center,
            radius=1000,
            color="red",
            fill=True,
            fill_opacity=0.3,
        ).add_to(city_map)

        # Display the map within the notebook
        display(city_map)

    else:
        print(f"Error fetching data for {city}. HTTP Status Code: {response.status_code}")

# Get user input for city
print("Select a city from the list:")
for city in cities.keys():
    print(f"- {city}")
city_name = input("Enter the city name: ").strip()

# AQI for the entered city
if city_name in cities:
    lat, lon = cities[city_name]
    get_aqi(city_name, lat, lon)
else:
    print(f"City '{city_name}' not found in the list. Please try again.")


Select a city from the list:
- Delhi
- Mumbai
- Kolkata
- Bangalore
- Chennai
- Hyderabad
- Ahmedabad
- Pune
- Jaipur
- Lucknow


Enter the city name:  Bangalore



City: Bangalore
Real-Time AQI: 158 (Poor) - Range: 151-200
Pollutant Concentrations (μg/m³):
CO: 574.11, NO2: 15.25, O3: 84.4
PM2.5: 69.52, PM10: 80.09, SO2: 4.95
